# Problem 1

> Describe the main components of a typical enigma machine.

# Problem 2

> What is the size of the key space?

In [24]:
from enigma.rotors.rotor import Rotor
from enigma.plugboard import Plugboard
from enigma.machine import EnigmaMachine

"""
During the war, Enigma machine operators re-configured their machines every day according to a code book, or key sheet, to help increase security. Each key sheet contained daily Enigma settings for one month. Before transmitting the first message of the day, the operator looked up the current day on the key sheet for the given month nd configured the machine accordingly.
"""

"""
When calling the Rotor constructor directly, the internal wiring is specified as a 26-character long string which specifies the cipher substitution. This notation is consistent with several online sources of Enigma information

`wiring` this should be a string of 26 uppercase characters A-Z that represent the internal wiring transformation of the signal as it enters from the right side. This is the format used in various online resources. For example, for the Wehrmacht Enigma type I rotor the mapping is "EKMFLGDQVZNTOWYHXUSPAIBRCJ"

`ring_setting` arguments are 0-based integers (0-25). his should be an integer from 0-25, inclusive, which indicates the Ringstellung. A value of 0 means there is no offset; e.g. the letter A is fixed to pin 0. A value of 1 means B is mapped to pin 0

`stepping` arguments specify when rotors turn their neighbor. This is the stepping or turnover parameter. When it is an iterable, for example a string such as “Q”, this indicates that when the rotor transitions from “Q” to “R” (by observing the operator window), the rotor will “kick” the rotor to its left, causing it to rotate.

If the rotor has more than one notch, a string of length 2 could be used, e.g. “ZM”. Another
way to think of this parameter is that when a character in the stepping string is visible in
the operator window, a notch is lined up with the pawl on the left side of the rotor. This
will allow the pawl to push up on the rotor and the rotor to the left when the next key is
depressed. A value of None means this rotor does not rotate.
"""
rL = Rotor('my rotor1', 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', ring_setting=0, stepping='Q')
rM = Rotor('my rotor2', 'BDFHJLCPRTXVZNYEIWGAKMUSQO', ring_setting=5, stepping='V')
rR = Rotor('my rotor3', 'ESOVPZJAYQUIRHXLNFTGKDCMWB', ring_setting=10, stepping='J')

"""
 Reflectors are simulated as rotors that have no ring setting or stepping capability
 If you decide to create your own reflector, and you desire to maintain reciprocal encryption & decryption (a fundamental characteristic of war-time Enigma machines), your connections must be made in pairs. Thus if you wire 'A' to 'G', you must also wire 'G' to 'A', and so on.
"""
reflector = Rotor('my reflector', 'YRUHQSLDPXNGOKMIEBFZCWVJAT')

"""
The plugboard, or Steckerbrett in German, allows the operator to swap up to 10 keys and indicator lamps for increased key strength. Plugboards have little use on their own.
"""
pb = Plugboard.from_key_sheet('AK BZ CG DL FU HJ MX NR OY PW')

#  the rotor assignment is specified by a list of rotors where order specifies the left-to-right order in the machine
machine = EnigmaMachine([rL, rM, rR], reflector, pb)

# must have one uppercase letter per rotor in your machine. In this example, we are setting the leftmost rotor to ‘U’, the middle rotor to ‘P’, and the rightmost rotor to ‘S’.
machine.set_display('UPS') # set initial rotor positions
print(machine.get_display()) # read rotor position

# Encrypt a letter
# print(machine.key_press('C'))
# Encrypt a text
# print(machine.process_text('Enigma machine is powerful for Q'))


UPS


In [59]:
from typing import Tuple
import re

MessageKey = str
EncryptionKey = str
CipherText = str
PlainText = str
EncryptionPayload = Tuple[EncryptionKey, CipherText]
DecryptionPayload = Tuple[MessageKey, PlainText]

three_uppercase_letters = re.compile(r'^[A-Z]{3}$')
is_three_uppercase_letters = lambda x: bool(three_uppercase_letters.search(x))

def encrypt(initial_rotor_position: str, payload: DecryptionPayload) -> EncryptionPayload:
    message_key, plaintext = payload

    assert(is_three_uppercase_letters(initial_rotor_position))
    assert(is_three_uppercase_letters(message_key))

    machine.set_display(initial_rotor_position)
    enc_key =  machine.process_text(message_key)

    machine.set_display(message_key)
    ciphertext = machine.process_text(plaintext)

    return (enc_key, ciphertext)

def decrypt(initial_rotor_position: str, payload: EncryptionPayload) -> DecryptionPayload:
    assert(is_three_uppercase_letters(initial_rotor_position))

    enc_key, ciphertext = payload
    assert(is_three_uppercase_letters(enc_key))

    machine.set_display(initial_rotor_position)
    message_key = machine.process_text(enc_key)

    assert(is_three_uppercase_letters(message_key))

    machine.set_display(message_key)

    plaintext = machine.process_text(ciphertext)
    return (message_key, plaintext)
    

encrypted_payload = encrypt('UPS', ('ABC', 'ENIGMA'))
decrypted_payload = decrypt('UPS', encrypted_payload)

assert(
    decrypted_payload == ('ABC', 'ENIGMA')
)


# Problem 3

> Explain the flow of the code and map the settings to the components mentioned in Problem 1. 

# Problem 4

> Learning by testing. Test the code with at least 7 various keys (change of wiring, ring_setting, stepping, reflector, with or without plugboard, initial display position, etc.

# Problem 5

>  Pick one of the ciphertext and decrypt it. Do we need to write another code for that?

# Problem 6

> Time for fun. Same setting as given in the sample code, but folks at Bletchley Park want to know the initial position (display). They had a ciphertext “WVUVJCSQBFLWSGTHDREWOSXYIAYEUBHHXY” from known plaintext “ATTACK AT 5PM AT ATLANTIC Z ISLAND”. Code and find the init display.

The message key is `NYU` and the initial rotor position is `UPS` as previous mentioned. There are also numerous other combinations which will successfully decrypt the same message.

In [60]:
from typing import Optional
from itertools import product
from string import ascii_uppercase

ciphertext = "WVUVJCSQBFLWSGTHDREWOSXYIAYEUBHHXY"
plaintext = "ATTACK AT 5PM AT ATLANTIC Z ISLAND"

initial_rotor_position: str = 'UPS'
message_key: Optional[MessageKey] = None

for enc_key in product(ascii_uppercase, repeat=3):
    decrypted_payload = decrypt(initial_rotor_position, (''.join(enc_key), ciphertext))
    if decrypted_payload[1].startswith('ATTACK'):
        message_key, _ = decrypted_payload
        print(decrypted_payload, initial_rotor_position, ''.join(enc_key))
        break

assert(message_key is not None)

encrypted_payload = encrypt(initial_rotor_position, (message_key, plaintext))
decrypted_payload = decrypt(initial_rotor_position, encrypted_payload)

assert(
    message_key == 'NYU' and decrypted_payload[0] == 'NYU'
)

assert(
    decrypted_payload[1] == 'ATTACKXATXXPMXATXATLANTICXZXISLAND'
)

assert(
    encrypt(initial_rotor_position, decrypted_payload) == encrypted_payload
)

('NYU', 'ATTACKXATXXPMXATXATLANTICXZXISLAND') UPS DSM


# Problem 7

> (optional) Same enigma machine given by the sample code. We had the ciphertext   “YNLIUNHBNVERXKRBUHZEYMJVEZNRPNWOSV” from known plaintext “ATTACK AT 5PM AT ATLANTIC Z ISLAND”. Recover as much of the stecker settings as it possible from the known plaintext.
